In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd

In [3]:
train_directory = 'C:/Users/Aravind/PROJECTS/Waste-Classification/Data/archive (2)/TRAIN'
test_directory = 'C:/Users/Aravind/PROJECTS/Waste-Classification/Data/archive (2)/TEST'

In [4]:
import tensorflow as tf

dataset = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    labels="inferred",
    label_mode="categorical",
    image_size=(128, 128),
    batch_size=32,
    shuffle=True,
    seed=42,
)

Found 20544 files belonging to 3 classes.


In [5]:
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    test_directory,
    labels="inferred",
    label_mode="categorical",
    image_size=(128, 128),
    batch_size=32,
    shuffle=True,
    seed=42,
)

Found 2528 files belonging to 3 classes.


In [6]:
from functools import partial

DefaultConv2D = partial(tf.keras.layers.Conv2D, kernel_size=3, strides=1,
                        padding="same", kernel_initializer="he_normal",
                        use_bias=False)

class ResidualUnit(tf.keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = tf.keras.activations.get(activation)
        self.main_layers = [
            DefaultConv2D(filters, strides=strides),
            tf.keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv2D(filters),
            tf.keras.layers.BatchNormalization()
        ]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv2D(filters, kernel_size=1, strides=strides),
                tf.keras.layers.BatchNormalization()
            ]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [7]:
model = tf.keras.Sequential([
    DefaultConv2D(64, kernel_size=7, strides=2, input_shape=[128, 128, 3]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"),
])
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters

model.add(tf.keras.layers.GlobalAvgPool2D())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(3, activation="softmax"))

In [8]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001 , rho=0.9)
model.compile(loss="categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
history = model.fit(
    dataset,
    validation_data=validation_dataset,
    epochs=1
)

KeyboardInterrupt: 

In [9]:
import pickle

with open('Pickle/dataset.pkl', 'wb') as file:
    pickle.dump(dataset, file)
with open('Pickle/validation_dataset.pkl', 'wb') as file:
    pickle.dump(validation_dataset, file)

InvalidArgumentError: Cannot convert a Tensor of dtype variant to a NumPy array.